In [1]:
import sys
sys.path.append('..')

In [2]:
from copy import deepcopy
import random
from library.problems.ks import KSSolution
from library.problems.data.ks_data import weights, values, capacity
from library.algorithms.genetic_algorithms.algorithm import genetic_algorithm
from library.algorithms.genetic_algorithms.selection import fitness_proportionate_selection

## Genetic operators

### Binary Standard Mutation

This mutation operator is used for binary string or list representations, such as '10001' or [1, 0, 0, 0, 1], found in problems like the Knapsack or IntBin problems.

Standard binary mutation works by iterating over each position (or gene) in the binary string. For each gene, there is a fixed mutation probability that determines whether the bit should be flipped (a 0 becomes 1 and vice versa)

![Binary Standard Mutation](images/binary-std-mutation.png)

Let's implement a function for standard binary mutation. It takes a binary representation and a mutation probability as inputs and returns a new representation.


In [3]:
def binary_standard_mutation(representation: str | list, mut_prob):
    """
    Applies standard binary mutation to a binary string or list representation.

    This function supports both binary strings (e.g., "10101") and binary lists 
    (e.g., [1, 0, 1, 0, 1]) containing either string characters ("0", "1") or 
    integers (0, 1). Each gene in the representation is independently flipped 
    with a given mutation probability, while preserving the original data type 
    of the genes.

    Parameters:
        representation (str or list): The binary representation to mutate.
        mut_prob (float): The probability of flipping each gene.

    Returns:
        str or list: A new mutated representation of the same type as the input.

    Raises:
        ValueError: If the input contains elements other than 0, 1, "0", or "1".
    """

    # Initialize new representation as a copy of current representation
    new_representation = deepcopy(representation)

    if random.random() <= mut_prob:
        # Strings are not mutable. Let's convert temporarily to a list
        if isinstance(representation, str):
            new_representation = list(new_representation)

        for char_ix, char in enumerate(representation):
            if char == "1":
                new_representation[char_ix] = "0"
            elif char == 1:
                new_representation[char_ix] = 0
            elif char == "0":
                new_representation[char_ix] = "1"
            elif char == 0:
                new_representation[char_ix] = 1
            else:
                raise ValueError(f"Invalid character {char}. Can not apply binary standard mutation")
    
        # If representation was a string, convert list back to string
        if isinstance(representation, str):
            new_representation = "".join(new_representation)

    return new_representation

Let's test on the Knapsack problem:

In [4]:
solution = KSSolution(values=values, weights=weights, capacity=capacity)

print("Solution:", solution)

new_solution_repr = binary_standard_mutation(solution.repr, mut_prob=0.2)

print("New solution:", new_solution_repr)

Solution: [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0]
New solution: [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0]


### Swap mutation

Swap mutation is a general-purpose operator suitable for any type of string or list-based representation.

It works by randomly selecting two positions (genes) in the solution and swapping their values. This swap is applied with a certain mutation probability.

![Swap Mutation](images/swap-mutation.png)

Let's implement the swap mutation function. It takes a representation and a mutation probability as inputs and returns a new solution where two genes may have been swapped.

In [5]:
def swap_mutation(representation, mut_prob):
    """
    Applies swap mutation to a solution representation with a given probability.

    Swap mutation randomly selects two different positions (genes) in the 
    representation and swaps their values. This operator is commonly used for 
    permutation-based representations but works for any list or string.

    The function preserves the type of the input representation: if the input is 
    a string, the output will also be a string; if it's a list, the output will 
    remain a list.

    Parameters:
        representation (str or list): The solution to mutate.
        mut_prob (float): The probability of performing the swap mutation.

    Returns:
        str or list: A new solution with two genes swapped, of the same type as the input.
    """

    # Strings are not mutable. Let's convert to list first
    if isinstance(representation, str):
        new_representation = deepcopy(list(representation))
    elif isinstance(representation, list):
        new_representation = deepcopy(representation)

    if random.random() <= mut_prob:
        first_idx = random.randint(0, len(representation) - 1)

        # To guarantee we select two different positions
        second_idx = first_idx
        while second_idx == first_idx:
            second_idx = random.randint(0, len(representation) - 1)

        new_representation[first_idx] = representation[second_idx]
        new_representation[second_idx] = representation[first_idx]


    # If representation was a string, convert list back to string
    if isinstance(representation, str):
        new_representation = "".join(new_representation)
    
    return new_representation

Now let's test on the Knapsack problem.

In [6]:
solution = KSSolution(values=values, weights=weights, capacity=capacity)

print("Solution:", solution)

new_solution_repr = swap_mutation(solution.repr, mut_prob=0.8)

print("New solution:", new_solution_repr)

Solution: [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
New solution: [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]


### Standard Crossover

Standard crossover takes two parent solutions, randomly selects a crossover point (an index between two consecutive genes) and exchanges the tail segments of the parents at that point. This process produces two new offspring that are combinations of their parents’ genetic material.

![Standard Crossover](images/std-crossover.png)

Let's implement the standard crossover function. It takes two parent representations as input and returns two offspring representations created by recombining segments from the parents at a randomly chosen crossover point.

In [ ]:
def standard_crossover(parent1_repr, parent2_repr):
    """
    Performs standard one-point crossover on two parent representations.

    This operator selects a random crossover point (not at the edges) and 
    exchanges the tail segments of the two parents to produce two offspring. 
    The crossover point is the same for both parents and ensures at least one 
    gene is inherited from each parent before and after the point.

    Parameters:
        parent1_repr (str or list): The first parent representation.
        parent2_repr (str or list): The second parent representation.
            Both parents must have the same length and type.

    Returns:
        tuple: A pair of offspring representations (offspring1, offspring2), 
        of the same type as the parents.

    Raises:
        ValueError: If parent representations are not the same length.
    """

    # Choose random crossover point
    xo_point = random.randint(1, len(parent1_repr) - 1)

    offspring1_repr = parent1_repr[:xo_point] + parent2_repr[xo_point:]
    offspring2_repr = parent2_repr[:xo_point] + parent1_repr[xo_point:]

    return offspring1_repr, offspring2_repr

Now let's test on the Knapsack problem.

In [8]:
parent1 = KSSolution(values=values, weights=weights, capacity=capacity)
parent2 = KSSolution(values=values, weights=weights, capacity=capacity)

print("Parent 1:", parent1)
print("Parent 2:", parent2)

offspring1_repr, offspring2_repr = standard_crossover(parent1.repr, parent2.repr)

print("Offspring 1:", offspring1_repr)
print("Offspring 2:", offspring2_repr)

Parent 1: [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1]
Parent 2: [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1]
Offspring 1: [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1]
Offspring 2: [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1]


## Solving Knapsack with Genetic Algorithms

![Knapsack Solutions](images/ks-solutions.png)

In the last notebook we implemented the genetic algorithm function. This function receives the following arguments:
- `initial_population`: List of individuals (randomly generated solutions)
- `max_gen`: Maximum number of generations
- `selection_algorithm`: A function that receives a population, selects one individual based on fitness and returns it
- `maximization`: Boolean that indicates if we're solving a maximization or minimization problem
- `xo_prob`: Probability of crossover (usually big)
- `mut_prob`: Probability of mutation (usually small)
- `elistism`: A boolean that indicates if elitism should be used or not

For this function to work, we need to comply with some assumptions
- individuals have `fitness`, `crossover` and `mutation` methods
- `crossover` always returns two offspring
- both `crossover` and `mutation` methods return new individuals instead of modifying individuals in-place

To solve the Knapsack Problem (KS) using this GA framework, we can define a new class, `KSGASolution`, which extends `KSSolution`. This allows us to inherit methods like `fitness`, `random_initial_representation`, and the `repr` attribute.

In `KSGASolution`, we'll implement the required `crossover` and `mutation` methods, adhering to the above assumptions.

For simplicity, let's use the standard crossover and binary standard mutation.

In [9]:
class KSGASolution(KSSolution):
    # Standard crossover
    def crossover(self, other_solution):
        # Choose random crossover point
        xo_point = random.randint(1, len(self.repr) - 2)

        offspring1_repr = self.repr[:xo_point] + other_solution.repr[xo_point:]
        offspring2_repr = other_solution.repr[:xo_point] + self.repr[xo_point:]

        return (
            KSGASolution(values=self.values, weights=self.weights, capacity=self.capacity, repr=offspring1_repr),
            KSGASolution(values=self.values, weights=self.weights, capacity=self.capacity, repr=offspring2_repr)
        )

    # Standard binary mutation
    def mutation(self, mut_prob):
        new_repr = deepcopy(self.repr)
        for char_ix, char in enumerate(self.repr):
            if char == 1:
                new_char = 0
            elif char == 0:
                new_char = 1
            else:
                raise ValueError("Invalid character. Can not apply standard mutation")
            
            if random.random() <= mut_prob:
                new_repr[char_ix] = new_char
        
        return KSGASolution(values=self.values, weights=self.weights, capacity=self.capacity, repr=new_repr)

Or we could just the functions we implemented in the beginning of the notebook!

In [10]:
class KSGASolution(KSSolution):
    # Standard crossover
    def crossover(self, other_solution):
        offspring1_repr, offspring2_repr = standard_crossover(self.repr, other_solution.repr)

        return (
            KSGASolution(values=self.values, weights=self.weights, capacity=self.capacity, repr=offspring1_repr),
            KSGASolution(values=self.values, weights=self.weights, capacity=self.capacity, repr=offspring2_repr)
        )

    # Standard binary mutation
    def mutation(self, mut_prob):
        new_repr = binary_standard_mutation(self.repr, mut_prob)

        return KSGASolution(values=self.values, weights=self.weights, capacity=self.capacity, repr=new_repr)

#### How can I test different crossover and mutation operators?

There are two approaches:
- Create separate classes for each combination of crossover and mutation operators.

    For example:
    - `KS_StdXO_StdMut_GASolution`
    - `KS_StdXO_SwapMut_GASolution`

    This method works but can quickly become repetitive and hard to maintain as the number of combinations grows.

- Make the solution class accept crossover and mutation functions as parameters during initialization.

    These functions would operate directly on the internal representation of the individual.
    ✅ This approach is much more modular and flexible!
    You can easily swap operators without needing to define new classes each time, making experimentation and tuning much easier.

In [11]:
class KSGASolution(KSSolution):
    def __init__(
        self,
        values,
        weights,
        capacity,
        mutation_function, # Callable
        crossover_function, # Callable
        repr = None
    ):
        super().__init__(
            values=values,
            weights=weights,
            capacity=capacity,
            repr=repr,
        )

        # Save as attributes for access in methods
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function

    
    def mutation(self, mut_prob):
        # Apply mutation function to representation
        new_repr = self.mutation_function(self.repr, mut_prob)
        # Create and return individual with mutated representation
        return KSGASolution(
            values=self.values,
            weights=self.weights,
            capacity=self.capacity,
            mutation_function=self.mutation_function,
            crossover_function=self.crossover_function,
            repr=new_repr
        )

    def crossover(self, other_solution):
        # Apply crossover function to self representation and other solution representation
        offspring1_repr, offspring2_repr = self.crossover_function(self.repr, other_solution.repr)

        # Create and return offspring with new representations
        return (
            KSGASolution(
                values=self.values,
                weights=self.weights,
                capacity=self.capacity,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring1_repr
            ),
            KSGASolution(
                values=self.values,
                weights=self.weights,
                capacity=self.capacity,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring2_repr
            )
        )

Let's test.

In [12]:
repr = [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1]

# Using std crossover and std mutation
solution1 = KSGASolution(
    values=values,
    weights=weights,
    capacity=capacity,
    mutation_function=binary_standard_mutation,
    crossover_function=standard_crossover,
    repr=repr
)


# Using std crossover and swap mutation
solution2 = KSGASolution(
    values=values,
    weights=weights,
    capacity=capacity,
    mutation_function=swap_mutation,
    crossover_function=standard_crossover,
    repr=repr
)

In [13]:
# Apply standard binary mutation
solution1.mutation(mut_prob=0.2)

[1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0]

In [14]:
# Apply standard swap mutation
solution2.mutation(mut_prob=0.2)

[0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1]

### Apply genetic algorithm

Let's run the genetic algorithm to solve Knapsack using standard crossover and standard binary mutation

In [ ]:
POP_SIZE = 50
initial_population = [
    KSGASolution(
        values=values,
        weights=weights,
        capacity=capacity,
        mutation_function=binary_standard_mutation,
        crossover_function=standard_crossover,
    )
    for _ in range(POP_SIZE)
]


best_solution = genetic_algorithm(
    initial_population=initial_population,
    selection_algorithm=fitness_proportionate_selection,
    max_gen=50,
    maximization=True,
    verbose=False,
    elitism=True,
)

print("Best solution:", best_solution)
print("Fitness:", best_solution.fitness())

Best solution: [1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
Fitness: 6012


And finally, let's apply the genetic algorithm again, but this time using swap mutation with a higher probability since it is not as disruptive as standard binary mutation.

In [ ]:
POP_SIZE = 50
initial_population = [
    KSGASolution(
        values=values,
        weights=weights,
        capacity=capacity,
        mutation_function=swap_mutation,
        crossover_function=standard_crossover,
    )
    for _ in range(POP_SIZE)
]


best_solution = genetic_algorithm(
    initial_population=initial_population,
    selection_algorithm=fitness_proportionate_selection,
    max_gen=50,
    mut_prob=0.8,
    maximization=True,
    verbose=False,
    elitism=True,
)

print("Best solution:", best_solution)
print("Fitness:", best_solution.fitness())

Best solution: [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1]
Fitness: 6772
